In [1]:
#!pip install linearmodels
#!pip install econtools
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


# table 3

**col2: pr& ctrl_exp**

In [2]:
#col2 final version
df_reg_pr = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_pr[df_reg_pr[o].isnull()==True].index
    df_name = df_reg_pr.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno'))

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			1355
R-squared:		0.5792
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  120
Fixed effects by:	authority_code
  No. of FE:		  18
               coeff    se      t   p>t  CI_low CI_high
fpsb_auction   8.688 1.101  7.892 0.000   6.509  10.868
reserve_price  0.000 0.000  8.644 0.000   0.000   0.000
OG02          -0.892 0.582 -1.532 0.128  -2.045   0.261
OG03          -1.436 0.405 -3.543 0.001  -2.239  -0.634
OG04          -5.529 3.406 -1.623 0.107 -12.273   1.216
OG06          -0.364 0.701 -0.519 0.604  -1.752   1.024
OG07           1.685 3.333  0.506 0.614  -4.914   8.284
OG08           3.017 1.905  1.584 0.116  -0.755   6.789
OG09           2.157 1.262  1.710 0.090  -0.341   4.655
OG10           7.825 0.476 16.431 0.000   6.882   8.768
OG11           5.187 0.870  5.963 0.000   3.465   6.909
OG12           4.093 2.431  1.684 0.095  -0.720   8.907
OG13          -3.047 1.265 -2.408 0.018  -5.553  -0.542
OS01 

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	delay_ratio
N:			1206
R-squared:		0.1263
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  117
Fixed effects by:	authority_code
  No. of FE:		  18
                coeff     se      t   p>t   CI_low CI_high
fpsb_auction   10.760 18.724  0.575 0.567  -26.326  47.845
reserve_price  -0.000  0.000 -2.619 0.010   -0.000  -0.000
OG02           -0.510  8.071 -0.063 0.950  -16.497  15.476
OG03           16.681  4.817  3.463 0.001    7.139  26.222
OG04          -14.647  7.047 -2.079 0.040  -28.604  -0.690
OG06           36.467 11.071  3.294 0.001   14.539  58.394
OG07            4.177 18.264  0.229 0.820  -31.998  40.352
OG08           15.130 14.197  1.066 0.289  -12.990  43.250
OG10           58.077 10.307  5.635 0.000   37.663  78.492
OG11            8.130 10.560  0.770 0.443  -12.785  29.045
OG12           77.504 52.608  1.473 0.143  -26.693 181.700
OG13           67.259  7.053  9.536 0.000   53.290  81.229
OS01           18.2

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	overrun_ratio
N:			1167
R-squared:		0.1488
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  117
Fixed effects by:	authority_code
  No. of FE:		  18
                coeff    se      t   p>t  CI_low CI_high
fpsb_auction    0.200 3.546  0.056 0.955  -6.824   7.224
reserve_price   0.000 0.000  2.747 0.007   0.000   0.000
OG02           -2.162 1.425 -1.517 0.132  -4.984   0.660
OG03           -2.257 1.180 -1.913 0.058  -4.594   0.079
OG04           -7.796 1.496 -5.212 0.000 -10.759  -4.833
OG06           -3.179 1.717 -1.852 0.067  -6.580   0.221
OG07           -2.429 7.237 -0.336 0.738 -16.763  11.904
OG08           -0.731 3.680 -0.199 0.843  -8.021   6.558
OG09          -10.413 3.307 -3.149 0.002 -16.962  -3.863
OG10           -5.472 2.964 -1.846 0.067 -11.343   0.399
OG11            1.251 2.372  0.528 0.599  -3.446   5.949
OG12           -2.864 5.673 -0.505 0.615 -14.099   8.372
OG13           -1.858 3.344 -0.556 0.580  -

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	days_to_award
N:			817
R-squared:		0.4089
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  106
Fixed effects by:	authority_code
  No. of FE:		  18
                coeff     se      t   p>t  CI_low CI_high
fpsb_auction   37.492  8.295  4.520 0.000  21.044  53.940
reserve_price   0.000  0.000  1.190 0.237  -0.000   0.000
OG02          -16.122  6.524 -2.471 0.015 -29.058  -3.185
OG03            0.371  3.377  0.110 0.913  -6.326   7.067
OG06           -2.176  5.958 -0.365 0.716 -13.991   9.638
OG07          -33.998 10.609 -3.205 0.002 -55.033 -12.963
OG08          -16.751  9.038 -1.853 0.067 -34.672   1.170
OG09          -14.941 11.778 -1.269 0.207 -38.294   8.411
OG10          -34.778  9.071 -3.834 0.000 -52.764 -16.792
OG11            8.179  8.709  0.939 0.350  -9.090  25.448
OG12           -6.275 12.025 -0.522 0.603 -30.119  17.569
OG13          -24.317 11.924 -2.039 0.044 -47.960  -0.675
OS01           -1.815 12.215 -0

# Col4. pr& ctrl_pop 

In [3]:
#col4 final version
df_reg_pr = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_pr[df_reg_pr[o].isnull()==True].index
    df_name = df_reg_pr.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno'))

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			653
R-squared:		0.6712
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  71
Fixed effects by:	authority_code
  No. of FE:		  11
               coeff    se      t   p>t  CI_low CI_high
fpsb_auction   8.294 1.166  7.111 0.000   5.968  10.620
reserve_price  0.000 0.000  5.806 0.000   0.000   0.000
OG02          -1.801 0.714 -2.521 0.014  -3.226  -0.376
OG03          -1.002 0.534 -1.876 0.065  -2.067   0.064
OG04          -1.190 0.643 -1.850 0.068  -2.473   0.093
OG06          -5.859 1.464 -4.002 0.000  -8.779  -2.939
OG07           1.224 3.414  0.359 0.721  -5.585   8.033
OG08           3.560 2.224  1.600 0.114  -0.877   7.997
OG10           7.896 0.738 10.704 0.000   6.425   9.367
OG11           5.918 1.112  5.323 0.000   3.701   8.135
OS01          -0.313 0.649 -0.482 0.631  -1.608   0.982
OS06           1.875 1.033  1.816 0.074  -0.185   3.935
OS07           1.154 0.873  1.323 0.190  -0.586   2.895
OS08   

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	delay_ratio
N:			560
R-squared:		0.1062
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  68
Fixed effects by:	authority_code
  No. of FE:		  11
                coeff     se      t   p>t   CI_low CI_high
fpsb_auction    7.987 18.273  0.437 0.663  -28.485  44.459
reserve_price  -0.000  0.000 -2.531 0.014   -0.000  -0.000
OG02           -1.905  9.925 -0.192 0.848  -21.715  17.905
OG03           17.389  9.410  1.848 0.069   -1.393  36.172
OG04          -23.950 11.583 -2.068 0.043  -47.069  -0.830
OG06           66.430 40.749  1.630 0.108  -14.906 147.766
OG07            7.856 18.901  0.416 0.679  -29.871  45.583
OG08           15.323 14.771  1.037 0.303  -14.161  44.806
OG10           63.417 10.401  6.097 0.000   42.657  84.176
OG11            3.648 13.978  0.261 0.795  -24.253  31.548
OS01           23.211 10.121  2.293 0.025    3.009  43.413
OS06           20.962 17.079  1.227 0.224  -13.127  55.052
OS07           53.156

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	overrun_ratio
N:			517
R-squared:		0.1882
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  67
Fixed effects by:	authority_code
  No. of FE:		  11
                coeff    se      t   p>t  CI_low CI_high
fpsb_auction    0.659 3.229  0.204 0.839  -5.787   7.106
reserve_price   0.000 0.000  1.555 0.125  -0.000   0.000
OG02           -4.059 2.438 -1.665 0.101  -8.927   0.810
OG03           -2.937 1.893 -1.552 0.126  -6.717   0.842
OG06           -3.665 3.171 -1.156 0.252  -9.997   2.666
OG07           -2.222 7.869 -0.282 0.779 -17.932  13.489
OG08           -1.459 4.561 -0.320 0.750 -10.566   7.648
OG10            0.272 2.397  0.113 0.910  -4.515   5.058
OG11           -3.207 2.743 -1.169 0.247  -8.682   2.269
OS01           -0.269 4.614 -0.058 0.954  -9.481   8.944
OS06           -0.605 1.821 -0.332 0.741  -4.241   3.031
OS08           12.062 4.355  2.770 0.007   3.367  20.757
OS10            4.422 6.803  0.650 0.518  -9.

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	days_to_award
N:			443
R-squared:		0.4340
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  63
Fixed effects by:	authority_code
  No. of FE:		  11
                coeff     se      t   p>t  CI_low CI_high
fpsb_auction   53.445  7.866  6.794 0.000  37.721  69.170
reserve_price   0.000  0.000  0.892 0.376  -0.000   0.000
OG02          -19.484  9.479 -2.055 0.044 -38.432  -0.536
OG03            6.352  6.409  0.991 0.326  -6.460  19.164
OG06            9.135 30.347  0.301 0.764 -51.529  69.798
OG07          -32.891 12.312 -2.672 0.010 -57.502  -8.281
OG08          -23.458  9.287 -2.526 0.014 -42.023  -4.894
OG10          -27.927  8.439 -3.309 0.002 -44.796 -11.057
OG11           15.341 10.880  1.410 0.164  -6.408  37.089
OS01          -19.784  9.678 -2.044 0.045 -39.130  -0.439
OS06           17.073 15.926  1.072 0.288 -14.762  48.909
OS07           49.039  8.485  5.780 0.000  32.078  66.000
OS08           13.615  6.686  2.

# Col 6. pr& ctrl_pop_exp: final version

In [134]:
#col6 setting
df_reg_pr = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_pr[df_reg_pr[o].isnull()==True].index
    df_name = df_reg_pr.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno'))

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			567
R-squared:		0.6752
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  52
Fixed effects by:	authority_code
  No. of FE:		  8
                coeff    se      t   p>t  CI_low CI_high
fpsb_auction    8.663 1.099  7.885 0.000   6.457  10.869
reserve_price   0.000 0.000  5.395 0.000   0.000   0.000
OG02           -2.206 0.774 -2.852 0.006  -3.760  -0.653
OG03           -0.864 0.593 -1.458 0.151  -2.054   0.326
OG04           -1.167 0.695 -1.680 0.099  -2.563   0.228
OG06           -5.835 1.452 -4.018 0.000  -8.750  -2.919
OG07            1.043 3.480  0.300 0.766  -5.945   8.030
OG08            3.524 2.219  1.588 0.118  -0.931   7.980
OG10            8.111 0.715 11.350 0.000   6.676   9.546
OG11            5.820 1.313  4.432 0.000   3.184   8.457
OS01           -0.446 0.590 -0.756 0.453  -1.630   0.738
OS06            1.573 1.178  1.335 0.188  -0.792   3.938
OS07            0.966 0.943  1.025 0.310  -0.926   

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	delay_ratio
N:			498
R-squared:		0.1058
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  50
Fixed effects by:	authority_code
  No. of FE:		  8
                coeff     se      t   p>t   CI_low CI_high
fpsb_auction    9.590 19.998  0.480 0.634  -30.598  49.778
reserve_price  -0.000  0.000 -2.714 0.009   -0.000  -0.000
OG02           -2.148 10.116 -0.212 0.833  -22.478  18.181
OG03           17.301 10.469  1.652 0.105   -3.739  38.340
OG04          -23.093 12.055 -1.916 0.061  -47.318   1.131
OG06           65.710 41.917  1.568 0.123  -18.526 149.946
OG07            9.569 19.415  0.493 0.624  -29.447  48.584
OG08           14.738 15.050  0.979 0.332  -15.505  44.981
OG10           63.865 10.839  5.892 0.000   42.085  85.646
OG11           10.237 14.265  0.718 0.476  -18.430  38.903
OS01           20.193 12.338  1.637 0.108   -4.600  44.986
OS06           18.912 18.580  1.018 0.314  -18.427  56.250
OS07           53.122 

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	overrun_ratio
N:			454
R-squared:		0.1926
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  50
Fixed effects by:	authority_code
  No. of FE:		  8
                coeff    se      t   p>t  CI_low CI_high
fpsb_auction    0.564 3.249  0.174 0.863  -5.965   7.093
reserve_price   0.000 0.000  1.228 0.225  -0.000   0.000
OG02           -2.947 2.306 -1.278 0.207  -7.581   1.687
OG03           -3.638 2.185 -1.665 0.102  -8.030   0.753
OG06           -3.792 3.220 -1.178 0.245 -10.264   2.680
OG07           -2.250 7.967 -0.282 0.779 -18.260  13.761
OG08           -1.678 4.744 -0.354 0.725 -11.211   7.855
OG10           -0.025 2.589 -0.010 0.992  -5.228   5.178
OG11           -0.796 2.429 -0.328 0.744  -5.677   4.084
OS01           -0.910 4.867 -0.187 0.853 -10.689   8.870
OS06           -0.240 2.112 -0.114 0.910  -4.483   4.004
OS08           11.452 4.302  2.662 0.010   2.807  20.097
OS10           15.300 3.011  5.082 0.000   9.2

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	days_to_award
N:			386
R-squared:		0.4732
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  49
Fixed effects by:	authority_code
  No. of FE:		  8
                coeff     se      t   p>t  CI_low CI_high
fpsb_auction   59.328  9.401  6.311 0.000  40.425  78.231
reserve_price  -0.000  0.000 -0.091 0.928  -0.000   0.000
OG02          -21.256 10.125 -2.099 0.041 -41.613  -0.898
OG03            4.633  6.803  0.681 0.499  -9.046  18.313
OG06            2.820 28.311  0.100 0.921 -54.104  59.743
OG07          -28.491 14.036 -2.030 0.048 -56.712  -0.271
OG08          -26.769  9.491 -2.820 0.007 -45.851  -7.686
OG10          -25.767  7.687 -3.352 0.002 -41.223 -10.310
OG11           10.551  9.399  1.122 0.267  -8.348  29.450
OS01          -31.325  8.309 -3.770 0.000 -48.032 -14.617
OS06           12.956 17.232  0.752 0.456 -21.691  47.603
OS07           44.311  8.799  5.036 0.000  26.620  62.002
OS08            5.755  9.475  0.6

# Odd columns

In [ ]:
#talbe 3 odds fianl version
#col1
df_reg_pr = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_pr[df_reg_pr[o].isnull()==True].index
    df_name = df_reg_pr.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)


    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)


In [ ]:
#talbe 3 odds fianl version
#col3
df_reg_pr = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_pr[df_reg_pr[o].isnull()==True].index
    df_name = df_reg_pr.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)


    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)


In [ ]:
#talbe 2 odds fianl version
#col5
df_reg_pr = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_pr[df_reg_pr[o].isnull()==True].index
    df_name = df_reg_pr.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)


    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)

#value chekced